In [49]:
import sys
import json
#if argv['params']['general'].get('scenario_path_S3') contains immense s3 path (ex: uuid-123-456-789/)
params = {'general':{'periods':['am','pm','ip','op']}}
default = {'scenario':'orchestrator','training_folder': '../..', 'params': params} # Default execution parameters
# here sceneario: demand is only use not on lambda.
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'scenario': 'orchestrator', 'training_folder': '../..', 'params': {'general': {'periods': ['am', 'pm', 'ip', 'op']}}}


In [50]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
import numpy as np

In [51]:
periods = argv['params']['general']['periods']
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)
training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'
if on_lambda:
    bucket_name = 'quetzal-immense' #os.environ.get('BUCKET_NAME')
    output_folder = training_folder + '/outputs/'
    scenario_folder = training_folder + '/inputs/'
    scenario_folders = []
    input_folders = []
    for p in periods:
        folder = 's3://' + bucket_name + '/' + p + '/outputs/'
        scenario_folders.append(folder)
        input_folders.append(input_folder + p + '/')

else:
    output_folder = training_folder + '/scenarios/' + argv['scenario'] + '/outputs/'
    scenario_folder = training_folder + '/scenarios/' + argv['scenario'] + '/inputs/'
    scenario_folders = []
    input_folders = []
    for p in periods:
        folder = training_folder + '/scenarios/' + p + '/outputs/'
        scenario_folders.append(folder)
        input_folders.append(input_folder + p + '/')
    
print('input folder: ', input_folders)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folders)

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

On Lambda :  False
input folder:  ['../../inputs/am/', '../../inputs/pm/', '../../inputs/ip/', '../../inputs/op/']
output folder:  ../../scenarios/orchestrator/outputs/
scen folder :  ['../../scenarios/am/outputs/', '../../scenarios/pm/outputs/', '../../scenarios/ip/outputs/', '../../scenarios/op/outputs/']


In [15]:
base_volumes = pd.read_csv(input_folder+'base/volumes/volumes.csv')
base_volumes.index.name = 'index'
volumes = base_volumes.copy()

In [16]:
volumes['origin'] = 'zone_' + volumes['origin'].astype(str)
volumes['destination'] = 'zone_' + volumes['destination'].astype(str)

In [17]:
od_set = set(zip(volumes['origin'],volumes['destination'],volumes['time_period'],volumes['vehicle_class']))

In [18]:
class_dict = {'C_other':'car',
            'C_EB':'car',
            'BPAX':'pt', 
            'RPAX':'pt'}
volumes['mode'] = volumes['vehicle_class'].apply(class_dict.get)


In [19]:
volumes = volumes.groupby(['origin','destination','time_period','mode'])[['volume']].agg(sum).unstack().reset_index()

In [20]:
volumes.columns = ['origin','destination','time_period','car','pt']

In [21]:
volumes = volumes.fillna(0)

In [22]:
period_dict={'am':'AM','pm':'PM','ip':'IP','op':'OP'}


In [23]:
#pour martin
cols = ['origin', 'destination', 'time_period', 'vehicle_class', 'time']
skim = pd.DataFrame()
ref_skim = pd.DataFrame()
for period, ref_folder, scen_folder in zip(periods,input_folders,scenario_folders):
    print(period)
    ref_pt_skim = pd.read_csv(scen_folder+'ref_pt_skim.csv')
    pt_skim = pd.read_csv(scen_folder+'pt_skim.csv')
    
    ref_pt_skim['time_period'] = period_dict[period]
    pt_skim['time_period'] = period_dict[period]

    ref_skim = pd.concat([ref_skim, ref_pt_skim[cols]])
    skim = pd.concat([skim, pt_skim[cols]])
    
    # skims pour martin:
    # origin, destination, time_period, vehicle_class, time,
    # ca: pt et road dans le meme fichier (vehicle_class => pt ou road)
    # ref et pas ref.

am
pm
ip
op


In [24]:
ref_car_skim = pd.read_csv(input_folder+ 'base/road_skims.csv')
car_skim = pd.read_csv(scenario_folder+'road_skims.csv')


In [25]:
ref_car_skim['origin'] = 'zone_' + ref_car_skim['origin'].astype(str)
ref_car_skim['destination'] = 'zone_' + ref_car_skim['destination'].astype(str)

car_skim['origin'] = 'zone_' + car_skim['origin'].astype(str)
car_skim['destination'] = 'zone_' + car_skim['destination'].astype(str)

In [26]:
if False: # export skims for martin
    cols = ['origin', 'destination', 'time_period', 'vehicle_class', 'time','distance']
    ref_skim = pd.concat([ref_skim, ref_car_skim[cols]])
    skim = pd.concat([skim, car_skim[cols]])

    skim = skim.set_index(['origin','destination','time_period','vehicle_class']).loc[od_set].reset_index()
    ref_skim = ref_skim.set_index(['origin','destination','time_period','vehicle_class']).loc[od_set].reset_index()

    skim.reset_index(drop=True).to_csv(output_folder+'skims'+'.csv')
    ref_skim.reset_index(drop=True).to_csv(output_folder+'ref_skims'+'.csv')

# modal shift

In [27]:
def get_pt_part(pt_los:pd.DataFrame, 
                car_los:pd.DataFrame, 
                method:str='kirchhoff',
                power:float=-1.42,
                part_max:float=1) -> pd.DataFrame:
    
    cols = ['origin','destination','time']
    los = pt_los[cols].merge(car_los[cols],on=['origin','destination'],how='inner')
    los = los.rename(columns = {'time_x':'time_pt','time_y':'time_car'})
    los['time_ratio'] = los['time_pt'] / los['time_car']
    if method == 'kirchhoff':
        los['pt_part'] = los['time_ratio'].apply(lambda x: min(x**(power), part_max))
    else:
        print('method not recognized')
    return los


In [28]:
for period, ref_folder, scen_folder in zip(periods,input_folders,scenario_folders):
    print(period)
    ref_pt_skim = pd.read_csv(scen_folder+'ref_pt_skim.csv')
    pt_skim = pd.read_csv(scen_folder+'pt_skim.csv')
    ref_car_los = ref_car_skim[ref_car_skim['time_period']== period_dict[period]]
    car_los = car_skim[car_skim['time_period']== period_dict[period]]
    
    v = volumes[volumes['time_period'] == period_dict[period]]
    
    ref_pt_part = get_pt_part(ref_pt_skim, ref_car_los)
    pt_part = get_pt_part(pt_skim, car_los)
    
    ref_pt_part = ref_pt_part.set_index(['origin','destination'])['pt_part'].to_dict()
    pt_part['ref_pt_part'] = pt_part.set_index(['origin','destination']).index.map(ref_pt_part.get)
    pt_part['transfert_prob'] = pt_part['pt_part'] - pt_part['ref_pt_part']
    
    transfert_prob = pt_part.set_index(['origin','destination'])['transfert_prob'].to_dict()
    v['transfert_prob'] = v.set_index(['origin','destination']).index.map(transfert_prob.get)
    v['transfert'] = v['pt'] * v['transfert_prob']
    
    volumes.loc[volumes['time_period'] == period_dict[period],'new_pt'] =v['pt'] + v['transfert']
    volumes.loc[volumes['time_period'] == period_dict[period],'new_car'] = v['car'] - v['transfert']
    
    #volumes[['origin','destination','pt','car']].to_csv(output_folder+'volume_'+period+'.csv')

am
pm
ip
op


In [29]:
volumes['new_car'] = volumes['new_car'].fillna(volumes['car'])
volumes['new_pt'] = volumes['new_pt'].fillna(volumes['pt'])

In [30]:
volumes['car_growth'] = volumes['new_car']/volumes['car']
volumes['pt_growth'] = volumes['new_pt']/volumes['pt']

In [31]:
base_volumes['o'] = 'zone_' + base_volumes['origin'].astype(str)
base_volumes['d'] = 'zone_' + base_volumes['destination'].astype(str)

In [32]:
car_dict = volumes.set_index(['origin','destination','time_period'])['car_growth'].to_dict()
pt_dict = volumes.set_index(['origin','destination','time_period'])['pt_growth'].to_dict()

In [33]:
base_volumes['pt_growth'] = base_volumes.set_index(['o','d','time_period']).index.map(pt_dict.get)
base_volumes['car_growth'] = base_volumes.set_index(['o','d','time_period']).index.map(car_dict.get)

In [34]:
car_filter = base_volumes['vehicle_class'].isin(['C_other', 'C_EB'])
base_volumes.loc[car_filter,'volume'] = base_volumes.loc[car_filter,'volume'] * base_volumes.loc[car_filter,'car_growth'] 


In [35]:
pt_filter = base_volumes['vehicle_class'].isin(['BPAX', 'RPAX'])
base_volumes.loc[pt_filter,'volume'] = base_volumes.loc[pt_filter,'volume'] * base_volumes.loc[pt_filter,'pt_growth'] 


In [36]:
base_volumes = base_volumes.drop(columns=['o','d','pt_growth','car_growth'])

# export

In [37]:
base_volumes.to_csv(output_folder+'volumes'+'.csv')

In [ ]:
# export to immense bucket

In [72]:
if argv['params']['general'].get('scenario_path_S3'):
    import boto3
    from io import StringIO

    s3_path = argv['params']['general'].get('scenario_path_S3')
    session = boto3.Session(region_name='eu-west-1')
    s3 = session.client('s3')
    
    csv_buffer = StringIO()
    base_volumes.to_csv(csv_buffer)

    s3.put_object(Body=csv_buffer.getvalue(),
                  Bucket='ir-dev-external-shared-eu-west-1-common', 
                  Key=s3_path+'outputs/volume.csv',
                  ACL='bucket-owner-full-control' )